# GPT Todo List

In [gpt-todo.ipynb](https://github.com/intellectronica/gpt-to-do/blob/main/gpt-todo.ipynb) we've implemented a to-do list that can be queried and modified using a chatbot. That implementation relied on LangChain's facilities for ReAct and tools. Since then, OpenAI released a new version of the GPT models that include external function calling as part of the API, and these models are now available on Azure Open AI. To put the new functionality to the test, I tried porting the example from LangChain to pure Open AI requests with function calls. As you'll see, I've been only partially successful. Even with this great new functionality, LangChain and similar libraries still have a lot to offer when it comes to orchestrating complex interactions with LLMs.

In [1]:
!pip install pandas openai


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [2]:
from IPython.display import display, HTML
from pandas import DataFrame, notnull

def display_tasks(tasks):
    "Display a list of tasks in a nice table"
    df = DataFrame(tasks)
    df.drop('id', axis=1, inplace=True)
    df['done'] = df['done'].apply(lambda x: '☑' if x else '☐')
    df['due'] = df['due'].apply(lambda x: x or '')
    df['priority'] = df['priority'].apply(lambda x: str(int(x)) if notnull(x) else '')
    display(HTML(df.to_html(index=False)))

We import the ToDoList object, and demonstrate a few simple actions by calling its methods directly and displaying the resulting list.

In [3]:
from todo import ToDoList
from datetime import date

todo_list = ToDoList()
task1 = todo_list.add("Buy milk", priority=1)
task2 = todo_list.add("Complete project", "2023-05-10", priority=2)
task3 = todo_list.add("Call wife", "2023-05-08")
task4 = todo_list.add("Buy groceries", "2023-05-09", priority=3)
task5 = todo_list.add("Put out fire", "2023-05-07", priority=0)
todo_list.update(task5['id'], done=True)
task6 = todo_list.add("Go for a walk", "2023-05-06", priority=3)
todo_list.update(task6['id'], done=True)


print("All tasks:")
display_tasks(todo_list.get())

print("Tasks sorted by due date:")
display_tasks(todo_list.get(sort='due'))

print("Tasks sorted by priority and then by due date:")
display_tasks(todo_list.get(sort=['priority', 'due']))

print("Priority 1 tasks:")
display_tasks(todo_list.get(priority=1))

print("Completed tasks:")
display_tasks(todo_list.get(done=True))

todo_list.delete(task2['id'])
print("Tasks after deleting task2:")
display_tasks(todo_list.get())

All tasks:


done,description,due,priority
☐,Buy milk,,1
☐,Complete project,2023-05-10,2
☐,Call wife,2023-05-08,
☐,Buy groceries,2023-05-09,3
☑,Put out fire,2023-05-07,0
☑,Go for a walk,2023-05-06,3


Tasks sorted by due date:


done,description,due,priority
☑,Go for a walk,2023-05-06,3
☑,Put out fire,2023-05-07,0
☐,Call wife,2023-05-08,
☐,Buy groceries,2023-05-09,3
☐,Complete project,2023-05-10,2
☐,Buy milk,,1


Tasks sorted by priority and then by due date:


...

done,description,due,priority
☐,Buy milk,,1


Completed tasks:


done,description,due,priority
☑,Put out fire,2023-05-07,0
☑,Go for a walk,2023-05-06,3


Tasks after deleting task2:


done,description,due,priority
☑,Put out fire,2023-05-07,0
☐,Buy milk,,1
☑,Go for a walk,2023-05-06,3
☐,Buy groceries,2023-05-09,3
☐,Call wife,2023-05-08,


Alright, time to create a user interface, which in this case means configuring a chatbot to interact with the to-do list on our behalf. We configure the Azure OpenAI gpt-3.5-turbo model, with the July release, which includes native function calling. We then define a function for each method ot the the to-do list, which we'll include in the calls to the LLM.

In [4]:
import os

os.environ['OPENAI_API_KEY'] = ' ... ' # Replace with the URL of an Azure OpenAI gpt-3.5-turbo deployment
os.environ['OPENAI_API_BASE'] = ' ... ' # Replace with the corresponding API key
os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['OPENAI_API_VERSION'] = '2023-07-01-preview'

deployment_name = 'gpt-35-turbo' # Replace if using a different deployment name

In [5]:
from typing import List, Optional, Dict, Union
import json

todo_functions = []

def todo_get(done: Optional[bool] = None, priority: Optional[int] = None, sort: Optional[Union[str, List[str]]] = None) -> str:
    """Read tasks from the todo list, whenever asked
    by the user about their tasks, or when you need to locate tasks by criteria.
    You can run the function with no parameters to get all tasks currently in the list,
    or pass one or more of the following parameters to filter the tasks: done - True or False,
    priority - 0, 1, 2, or 3,  sort - 'due', 'priority', or ['priority', 'due'].
    """
    tasks = todo_list.get(done, priority, sort)
    return "Tasks:\n" + json.dumps(tasks, indent=2)

todo_functions.append({
    'name': 'todo_get',
    'description': todo_get.__doc__,
    'parameters':{
        'type': 'object',
        'properties': {
            'done': {
                'type': 'boolean',
                'description': 'is the task marked as done?',
            },
            'priority': {
                'type': 'integer',
                'description': 'the priority of the task',
            },
            'sort': {
                'oneOf': [
                    {'type': 'string'},
                    {'type': 'array', 'items': {'type': 'string'}},
                ],
                'description': (
                    'the sort order of the tasks as the string "due" or "priority", '
                    'or a list of "due" and/or "priority'
                ),
            },
        },
    }
})

def todo_add(description: str, due: Optional[str] = None, priority: Optional[int] = None) -> str:
    """Add a task to the todo list, whenever requested by the user.
    You must provide a description for the task, and optionally a due date and a priority.
    """
    task = todo_list.add(description, due, priority)
    return "Added task:\n" + json.dumps(task, indent=2)

todo_functions.append({
    'name': 'todo_add',
    'description': todo_add.__doc__,
    'parameters':{
        'type': 'object',
        'properties': {
            'description': {
                'type': 'string',
                'description': 'date when the task is due, in the format YYYY-MM-DD',
            },
            'due': {
                'type': 'string',
                'description': 'the sort order of the tasks, comma-separated list of "due" and/or "priority"',
            },
            'priority': {
                'type': 'integer',
                'description': 'the priority of the task',
            },
        },
    }
})

def todo_delete(task_id: str) -> str:
    """Delete a task from the todo list, whenever requested by the user.
    You must provide the ID of the task to delete.
    If you are not sure of the ID, you can use the todo_get tool to locate the tasks
    you want to delete and read their IDs.
    """
    result = todo_list.delete(task_id)
    return "Result:\n" + json.dumps({'deleted': result}, indent=2)

todo_functions.append({
    'name': 'todo_delete',
    'description': todo_delete.__doc__,
    'parameters':{
        'type': 'object',
        'properties': {
            'task_id': {
                'type': 'string',
                'description': 'ID of the task to delete',
            },
        },
    }
})

def todo_update(task_id: str, done: Optional[bool] = None, description: Optional[str] = None,
                due: Optional[str] = None, priority: Optional[int] = None) -> str:
    """Modify a task in the todo list, whenever requested by the user.
    You must provide the ID of the task to update, and optionally any of the following
    parameters to modify the task: done - True or False, description - a new description
    for the task, due - a new due date for the task, priority - a new priority for the task

    If you are not sure of the ID, you can use the `todo_get` function to locate the tasks
    you want to update and read their IDs.
    """
    task = todo_list.update(task_id, done, description, due, priority)
    return "Updated task:\n" + json.dumps(task, indent=2)

todo_functions.append({
    'name': 'todo_update',
    'description': todo_update.__doc__,
    'parameters':{
        'type': 'object',
        'properties': {
            'task_id': {
                'type': 'string',
                'description': 'ID of the task to delete',
            },
            'done': {
                'type': 'boolean',
                'description': 'is the task marked as done?',
            },
            'description': {
                'type': 'string',
                'description': 'date when the task is due, in the format YYYY-MM-DD',
            },
            'due': {
                'type': 'string',
                'description': 'the sort order of the tasks, comma-separated list of "due" and/or "priority"',
            },
            'priority': {
                'type': 'integer',
                'description': 'the priority of the task',
            },
        },
    }
})

In [6]:
import openai, json

def todo_chat(question):
  "Chat with the todo-list agent and display the result and the current state of the to-do list."
  messages = [
    {'role': 'system', 'content': (
      'You are a todo-list agent. You maintain a list of tasks that you can query, add, '
      'or modify, using function calls. '
      'The user will ask questions about their task list, or ask you to make changes to it. '
      'Use functions to interact with the list and follow the user\'s instructions. '
      'Some requests from the user may require multiple function calls. For example, '
      'You may need to call the `todo_get` function to get the list of tasks with their IDs, '
      'then call the `todo_delete` function to delete a task by ID or `todo_update` to '
      'modify a task.'
    )},
    {'role': 'user', 'content': question},
  ]
  response = openai.ChatCompletion.create(
    engine=deployment_name,
    messages=messages,
    functions=todo_functions,
    function_call="auto", 
  )
  response_message = response['choices'][0]['message']
  while response_message.get('function_call'):
    print('{function_name}({function_args})'.format(
      function_name=response_message['function_call']['name'],
      function_args=json.loads(response_message['function_call']['arguments'])
    ))
    func = globals()[response_message['function_call']['name']]
    args = json.loads(response_message['function_call']['arguments'])
    retval = func(**args)
    messages.append({
      'role': 'assistant',
      'name': response_message['function_call']['name'],
      'content': response_message['function_call']['arguments'],
    })
    messages.append({
      'role': 'function',
      'name': response_message['function_call']['name'],
      'content': retval,
    })
    response = openai.ChatCompletion.create(
      engine=deployment_name,
      messages=messages,
    )
    response_message = response['choices'][0]['message']
  answer = response_message['content']
  print('Question: ' + question)
  print('Answer:')
  print(answer)
  print('Debug:')
  display_tasks(todo_list.get())  

Cool, we've got everything ready. Let's make a few requests to the to-do list agent to see it in action!

In [7]:
todo_chat("Add a task to do the dishes today.")

todo_add({'description': 'do the dishes', 'due': 'today'})
Question: Add a task to do the dishes today.
Answer:
I have added a task to do the dishes today.
Debug:


done,description,due,priority
☑,Put out fire,2023-05-07,0
☐,Buy milk,,1
☑,Go for a walk,2023-05-06,3
☐,Buy groceries,2023-05-09,3
☐,Call wife,2023-05-08,
☐,do the dishes,2023-07-28,


In [8]:
todo_chat("Find the id of the task 'Buy milk' and update it to mark it as complete.")

todo_get({})
Question: Find the id of the task 'Buy milk' and update it to mark it as complete.
Answer:
The ID of the task 'Buy milk' is c962df6a-bcdb-4b0b-9c6e-a92ad56e01b7. Let me mark it as complete.
Debug:


done,description,due,priority
☑,Put out fire,2023-05-07,0
☐,Buy milk,,1
☑,Go for a walk,2023-05-06,3
☐,Buy groceries,2023-05-09,3
☐,Call wife,2023-05-08,
☐,do the dishes,2023-07-28,


(!!!) FAIL : when a request requires multiple function calls, the assistant responds with the first request, and then doesn't call the next function, so the request is not fulfilled.

In [9]:
todo_chat("What are all the tasks in the list that are incomplete?")

todo_get({'done': False})
Question: What are all the tasks in the list that are incomplete?
Answer:
The tasks that are incomplete are:

1. Buy milk
2. Buy groceries
3. Call wife
4. Do the dishes
Debug:


done,description,due,priority
☑,Put out fire,2023-05-07,0
☐,Buy milk,,1
☑,Go for a walk,2023-05-06,3
☐,Buy groceries,2023-05-09,3
☐,Call wife,2023-05-08,
☐,do the dishes,2023-07-28,


In [10]:
todo_chat("Which tasks have priority 3?")

todo_get({'priority': 3})
Question: Which tasks have priority 3?
Answer:
The tasks with priority 3 are:

1. Task ID: 2ecc52e1-65db-47cc-abec-74430307caa1
   Description: Go for a walk
   Due date: 2023-05-06
   Status: Done

2. Task ID: 29d5f622-5022-4899-88ab-3fa567bdcae7
   Description: Buy groceries
   Due date: 2023-05-09
   Status: Not done
Debug:


done,description,due,priority
☑,Put out fire,2023-05-07,0
☐,Buy milk,,1
☑,Go for a walk,2023-05-06,3
☐,Buy groceries,2023-05-09,3
☐,Call wife,2023-05-08,
☐,do the dishes,2023-07-28,


In [11]:
todo_chat("Delete a task that is complete and has priority 0.")

todo_get({'done': True, 'priority': 0})
Question: Delete a task that is complete and has priority 0.
Answer:
The only task that is complete and has priority 0 is the task with the ID "ee6370d2-44fd-4a3e-8bf9-ebbe99699f85". Would you like me to delete this task?
Debug:


done,description,due,priority
☑,Put out fire,2023-05-07,0
☐,Buy milk,,1
☑,Go for a walk,2023-05-06,3
☐,Buy groceries,2023-05-09,3
☐,Call wife,2023-05-08,
☐,do the dishes,2023-07-28,


(!!!) FAIL : once again, when a request requires multiple function calls, the assistant responds after the first one and doesn't continue to execute the second function, and the original request isn't completed.

In summary: function calling is easy to use and works accurately, making it easy to get GPT to make use of external functions. Where things get difficult is when we need to orchestrate complex, multi-step, interactions. Without the support of an orchestration library with an implementation of a chaining technique like ReAct, we are limited to single-shot interactions (despite some minimal attempts to instruct the model using the system message).